## 5장 분할
분할은 예측 모델을 평가할 때 필요한 전처리 입니다. 머신러닝에서 지도학습은 학습 데이터(예측 모델 구축용 데이터)와 검증 데이터(정확도 측정을 위한 예측을 할 데이터)가 필요합니다. 각각 두개의 데이터는 본질적으로 같습니다. 따라서 전처리를 할때 묶어서 다루고, 입력하기 직전에 분할하는 것이 어느정도 효율적이라고 생각할 수 있습니다.

# 모델 검증을 위한 데이터 레코드 분할
분할이 자주 사용되는 곳은 모델 검증을 위한 데이터 레코드 분할입니다.
이중 검증 방법에는 교차 검증과 홀드아웃 검증이 있습니다.

우선 교차 검증을 위한 데이터 분할을 살펴보겠습니다.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

production_tb = pd.read_csv('production.csv')
production_tb

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(production_tb.drop('fault_flg', axis = 1),
                                                                    production_tb[['fault_flg']], test_size = 0.2)

In [ ]:
train_data.reset_index(inplace = True, drop = True)
train_data

In [ ]:
test_data.reset_index(inplace=True, drop = True)
train_target.reset_index(inplace = True, drop = True)
test_target.reset_index(inplace = True, drop = True)

train_test_split에 의해 학습용 데이터와 검증용 데이터가 8:2 비율로 구분되었습니다
- 전체 row 개수는 1000개 인데, 학습용은 0~799 즉 800개 입니다.

그리고 행 이름을 현재의 행번호로 reset_index함수를 통해 바꾸었습니다.

In [ ]:
row_no_list = list(range(len(train_target)))

In [ ]:
k_fold = KFold(n_splits=4, shuffle=True)

In [ ]:
for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]

데이터를 분할할때, 랜덤하게 나열하므로 index가 섞이는데, reset_index를 통해 다시 재정렬 시켜줍니다.
여기서 KFold 함수에서 데이터 행이 랜덤하게 나열되지 않는 경우 shuffle = True를 통해 섞어주어야 합니다.

# 모델 검증을 위한 시간 데이터 분할
시간 데이터를 이용한 검증은 과거의 데이터에서 예측 모델을 만들고, 미래의 가격을 예측해야 합니다.
그러나 교차 검증을 이용하여 분할할 경우 과거와 미래의 데이터가 섞입니다.
이를 어느정도 보완한 방법이 시간축을 기준으로 이동하면서 검증하는 방법입니다.
첫번째 검증은 1-6월 데이터를 통해 학습, 7-8월 데이터를 검증.
두번째 검증은 3-8월 데이터를 통해 학습, 9-10월 데이터를 검증
세번째 검증은 5-10월 데이터를 통해 학습, 11-12월 데이터를 검증 이런 방식입니다.
그러나 거시적인 변화가 큰 경우 학습데이터를 늘리는 방법도 있습니다.

In [ ]:
train_window_start = 1
train_window_end = 24
horizon = 12
skip = 12

train_window_start, end는 각각 맨 처음 학습 데이터의 시작과 끝의 행 번호를 지정합니다.
horizon은 검정 데이터의 데이터 수를 지정한 것이고, skip은 이동하는 데이터 수를 설정한 것입니다.

In [ ]:
monthly_index_tb = pd.read_csv('monthly_index.csv')
monthly_index_tb

In [ ]:
monthly_index_tb.sort_values(by = 'year_month')
monthly_index_tb

In [ ]:
while True:
    test_window_end = train_window_end + horizon
    # 검증 데이터의 종료 행번호를 계산합니다. 여기서 horizon이 검정 데이터의 개수이므로, 테스트 데이터에 horizon을 더하면 됩니다.
    train = monthly_index_tb[train_window_start:train_window_end]
    # 여기서 아까 설명에서 검증하는 데이터의 날짜가 이동하는데, 만약에 데이터를 늘릴경우 train_window_start = 1을 하면 됩니다.
    test = monthly_index_tb[(train_window_end + 1):test_window_end]
    
    if test_window_end >= len(monthly_index_tb.index):
        break
    # 검증 데이터의 인덱스가 전체의 인덱스 번호가 크다는 뜻은 검증을 다 마쳤다는 뜻으로, 다 마쳤으면 break 입니다.
    train_window_start += skip
    train_window_end += skip
    # 인덱스에 skip 즉 이동시킬 데이터 인덱스 만큼 더함으로써, 데이터를 이동시킵니다.